<a href="https://colab.research.google.com/github/mohammad-nour-alawad/LLM-Voice-to-Pandas/blob/main/Voice_Interpreter_of_Natural_Language_into_Pandas_Commands.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.4 MB/s eta 0:00:00


In [ ]:
import json
import pandas as pd
from tqdm import tqdm
from google.colab import drive
import zipfile
import os

# Define the path to the zip file and extraction directory
zip_file_path = '/content/drive/MyDrive/spider-linux.zip'
extraction_dir = '/content/spider'

# Unzip the dataset
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extraction_dir)

# Define the paths to the Spider dataset JSON files
train_file_path = os.path.join(extraction_dir, 'spider-linux/train_spider.json')
dev_file_path = os.path.join(extraction_dir, 'spider-linux/dev.json')

# Load the Spider dataset JSON file
def load_spider_json(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

# Extract the text queries and SQL commands
def extract_queries_and_commands(data):
    queries_and_commands = []
    for item in data:
        question = item['question']
        sql = item['query']
        queries_and_commands.append({'text_query': question, 'sql_command': sql})
    return queries_and_commands

# Main function to process the Spider dataset and save to CSV
def main():
    # Load the data
    train_data = load_spider_json(train_file_path)
    dev_data = load_spider_json(dev_file_path)

    # Extract the queries and commands
    train_queries_and_commands = extract_queries_and_commands(train_data)
    dev_queries_and_commands = extract_queries_and_commands(dev_data)

    # Print the number of rows for each part
    print(f"Number of rows in training set: {len(train_queries_and_commands)}")
    print(f"Number of rows in development set: {len(dev_queries_and_commands)}")

    # Combine train and dev data
    all_queries_and_commands = train_queries_and_commands + dev_queries_and_commands

    # Print the total number of rows
    print(f"Total number of rows: {len(all_queries_and_commands)}")

    # Convert to DataFrame
    df = pd.DataFrame(all_queries_and_commands)

    # Save to CSV in Google Drive
    output_file_path = '/content/spider_text_sql.csv'
    df.to_csv(output_file_path, index=False)

    print(f"Data has been saved to {output_file_path}")

# Run the main function
if __name__ == "__main__":
    main()


Number of rows in training set: 7000
Number of rows in development set: 1034
Total number of rows: 8034
Data has been saved to /content/spider_text_sql.csv


# Get the CSV data.

# Example

In [ ]:
import pandas as pd
from tqdm import tqdm
from openai import OpenAI

# Initialize the OpenAI client with your API key
client = OpenAI(api_key="sk-b444e209aff54dd9a8e6013363357551", base_url="https://api.deepseek.com/v1")

# Function to get Pandas code for a given SQL query
def get_pandas_code(row):
    sql_query = row["sql_query"]

    messages = [
        {"role": "system", "content": "please do not discuss or explain or justify, just convert the SQL command into one single python Pandas command (with no imports and no comments): "},
        {"role": "user", "content": f"SQL Query: {sql_query}"}
    ]

    try:
        response = client.chat.completions.create(
            model="deepseek-chat",
            messages=messages
        )
        return response.choices[0].message.content.strip()

    except Exception as e:
        print(f"Error processing SQL query: {sql_query}: {str(e)}")
        return "Error converting query"

# Example DataFrame with SQL queries
data = {
    "sql_query": [
        "SELECT * FROM your_table",
        "SELECT name, age FROM your_table WHERE age > 30",
        "SELECT COUNT(*) FROM your_table"
    ]
}

df_sql = pd.DataFrame(data)

# Apply the get_pandas_code function to each row in the DataFrame
df_sql['pandas_code'] = [get_pandas_code(row) for _, row in tqdm(df_sql.iterrows(), total=df_sql.shape[0])]


print("\nConversion results have been saved to sql_to_pandas_results.csv")

print(df_sql)

# df_sql.to_csv('sql_to_pandas_results.csv', index=False)




100%|██████████| 3/3 [00:05<00:00,  1.91s/it]

Conversion results have been saved to sql_to_pandas_results.csv


# Conversion of Spider dataset to **Text-to-Pandas**


In [ ]:
import pandas as pd
from tqdm import tqdm
from openai import OpenAI

# Initialize the OpenAI client with your API key
client = OpenAI(api_key="sk-b444e209aff54dd9a8e6013363357551", base_url="https://api.deepseek.com/v1")

def get_pandas_code(row):
    sql_query = row["sql_command"]

    messages = [
        {"role": "system", "content": "please do not discuss or explain or justify, just convert \
                                       the SQL command into one single python Pandas command \
                                       (with no imports and no comments):"},
        {"role": "user", "content": f"SQL Query: {sql_query}"}
    ]

    try:
        response = client.chat.completions.create(
            model="deepseek-chat",
            messages=messages
        )
        return response.choices[0].message.content.strip()

    except Exception as e:
        print(f"Error processing SQL query: {sql_query}: {str(e)}")
        return "Error converting query"

# Load the CSV file containing text queries and SQL commands
input_file_path = '/content/spider_text_sql.csv'  # Adjust the path as per your Kaggle dataset location
df_sql = pd.read_csv(input_file_path)

# Define the chunk size
chunk_size = 1000

# Process and save in chunks
for start in range(4000, df_sql.shape[0], chunk_size):
    end = start + chunk_size
    df_chunk = df_sql.iloc[start:end].copy()
    df_chunk['pandas_code'] = [get_pandas_code(row) for _, row in tqdm(df_chunk.iterrows(), total=df_chunk.shape[0])]

    # Define the output file path for each chunk
    output_file_path = f'/content/sql_to_pandas_results_chunk_{start//chunk_size + 1}.csv'
    df_chunk.to_csv(output_file_path, index=False)

    print(f"Conversion results for rows {start} to {end - 1} have been saved to {output_file_path}")

print("All chunks have been processed and saved.")


## Merge the chuncks

In [ ]:
import pandas as pd
import glob

# Define the file paths
folder_path = "/content/drive/MyDrive/NIR/SPIDER PANDAS/"
file_pattern = folder_path + "sql_to_pandas_results_chunk_*.csv"

# Use glob to get all file paths matching the pattern
file_paths = glob.glob(file_pattern)

# Read and concatenate all CSV files
dfs = [pd.read_csv(file_path) for file_path in file_paths]
merged_df = pd.concat(dfs, ignore_index=True)

# Save the merged DataFrame back to the folder
output_file_path = folder_path + "merged_sql_to_pandas_results.csv"
merged_df.to_csv(output_file_path, index=False)

print(f"Merged file saved to {output_file_path}")


Merged file saved to /content/drive/MyDrive/NIR/SPIDER PANDAS/merged_sql_to_pandas_results.csv


## FIX THE ERRORS

In [ ]:
import pandas as pd
from tqdm import tqdm
from openai import OpenAI

# Initialize the OpenAI client with your API key
client = OpenAI(api_key="sk-b444e209aff54dd9a8e6013363357551", base_url="https://api.deepseek.com/v1")

# Function to get Pandas code for a given SQL query
def get_pandas_code(row):
    sql_query = row["sql_command"]

    messages = [
        {"role": "system", "content": "please do not discuss or explain or justify, just convert the SQL command into one single python Pandas command (with no imports and no comments):"},
        {"role": "user", "content": f"SQL Query: {sql_query}"}
    ]

    try:
        response = client.chat.completions.create(
            model="deepseek-chat",
            messages=messages
        )
        return response.choices[0].message.content.strip()

    except Exception as e:
        print(f"Error processing SQL query: {sql_query}: {str(e)}")
        return "Error converting query"

folder_path = "/content/drive/MyDrive/NIR/SPIDER PANDAS/"
output_file_path = folder_path + "merged_sql_to_pandas_results.csv"

# Load the merged CSV file
merged_df = pd.read_csv(output_file_path)

# Only process rows where pandas_code is "Error converting query"
error_df = merged_df[merged_df['pandas_code'] == "Error converting query"].copy()

# Apply the function to these rows
error_df['pandas_code'] = [get_pandas_code(row) for _, row in tqdm(error_df.iterrows(), total=error_df.shape[0])]

# Update the original DataFrame with the corrected codes
merged_df.update(error_df)

# Save the updated DataFrame back to the folder
merged_output_file_path = folder_path + "updated_merged_sql_to_pandas_results.csv"
merged_df.to_csv(merged_output_file_path, index=False)

print(f"Updated merged file saved to {merged_output_file_path}")


100%|██████████| 52/52 [02:59<00:00,  3.46s/it]

Updated merged file saved to /content/drive/MyDrive/NIR/SPIDER PANDAS/updated_merged_sql_to_pandas_results.csv


## clean codes headers  

In [ ]:
import pandas as pd
import re

# Define the folder path and file paths
folder_path = "/content/drive/MyDrive/NIR/SPIDER PANDAS/"
input_file_path = folder_path + "updated_merged_sql_to_pandas_results.csv"
merged_output_file_path = folder_path + "updated_merged_sql_to_pandas_results.csv"

# Function to clean pandas_code column
def clean_pandas_code(code):
    # Remove ```python and ``` if they exist
    return re.sub(r'```python\n|```', '', code).strip()

# Read the input CSV file into a DataFrame
df = pd.read_csv(input_file_path)

# Apply the cleaning function to the pandas_code column
df['pandas_code'] = df['pandas_code'].apply(clean_pandas_code)

# Write the cleaned DataFrame to a new CSV file
df.to_csv(merged_output_file_path, index=False)

print("Pandas code cleaned and updated file saved to:", merged_output_file_path)


Pandas code cleaned and updated file saved to: /content/drive/MyDrive/NIR/SPIDER PANDAS/updated_merged_sql_to_pandas_results.csv


# TRAIN LLM

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install transformers[torch] datasets accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.w

## Load Your Dataset

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset

# Load the CSV file into a DataFrame
file_path = '/content/drive/MyDrive/NIR/SPIDER PANDAS/updated_merged_sql_to_pandas_results.csv'
data = pd.read_csv(file_path)

# Clean the pandas_code column
data['pandas_code'] = data['pandas_code'].str.replace('```python\n', '').str.replace('\n```', '').str.replace('\n', '')

# Split the dataset into training and evaluation sets
train_data, eval_data = train_test_split(data, test_size=0.2, random_state=42)

# Convert the DataFrames to Hugging Face Datasets
train_dataset = Dataset.from_pandas(train_data[['text_query', 'pandas_code']])
eval_dataset = Dataset.from_pandas(eval_data[['text_query', 'pandas_code']])


# T5 Model

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset

# Load the CSV file into a DataFrame
file_path = '/content/drive/MyDrive/NIR/SPIDER PANDAS/updated_merged_sql_to_pandas_results.csv'
data = pd.read_csv(file_path)

# Clean the pandas_code column
data['pandas_code'] = data['pandas_code'].str.replace('```python\n', '').str.replace('\n```', '').str.replace('\n', '')

# Split the dataset into training and evaluation sets
train_data, eval_data = train_test_split(data, test_size=0.2, random_state=42)

# Convert the DataFrames to Hugging Face Datasets
train_dataset = Dataset.from_pandas(train_data[['text_query', 'pandas_code']])
eval_dataset = Dataset.from_pandas(eval_data[['text_query', 'pandas_code']])


from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load the tokenizer and model for T5
model_name = "t5-small"  # You can also try "t5-base" or "t5-large"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Tokenize the dataset with a higher max_length
def preprocess_function(examples):
    inputs = examples['text_query']
    targets = examples['pandas_code']
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")

    # Set up the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=512, truncation=True, padding="max_length")

    model_inputs['labels'] = labels['input_ids']
    return model_inputs

tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True)
tokenized_eval_dataset = eval_dataset.map(preprocess_function, batched=True)

from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

# Set up training arguments with appropriate max length
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    eval_accumulation_steps=10,
    generation_max_length=512,
)

# Initialize the Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset,
)

# Train the model
trainer.train()

# Evaluate the model
eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3946: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/1607 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,1.355200,0.224151
2,0.244100,0.179506
3,0.223200,0.169730


Evaluation results: {'eval_loss': 0.16972976922988892, 'eval_runtime': 44.7415, 'eval_samples_per_second': 35.917, 'eval_steps_per_second': 4.492, 'epoch': 3.0}


In [ ]:
# Save the model
model.save_pretrained("trained_model_t5")
tokenizer.save_pretrained("trained_model_t5")

# Download the model files (optional)
!zip -r trained_model_t5.zip trained_model_t5
from google.colab import files
files.download("trained_model_t5.zip")


  adding: trained_model_t5/ (stored 0%)
  adding: trained_model_t5/config.json (deflated 62%)
  adding: trained_model_t5/model.safetensors (deflated 11%)
  adding: trained_model_t5/added_tokens.json (deflated 83%)
  adding: trained_model_t5/spiece.model (deflated 48%)
  adding: trained_model_t5/special_tokens_map.json (deflated 85%)
  adding: trained_model_t5/tokenizer_config.json (deflated 94%)
  adding: trained_model_t5/generation_config.json (deflated 29%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import torch

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Move the model to the device
model.to(device)


Using device: cuda


T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

## Generate Predictions

In [ ]:
# Generate predictions for a subset of the evaluation dataset
import numpy as np
from tqdm import tqdm

# Adjust generation parameters
generation_args = {
    "max_length": 512,
    "num_beams": 5,
    "early_stopping": True,
    "no_repeat_ngram_size": 2,
}

def generate_predictions(eval_dataset, model, tokenizer, device, num_samples=100):
    eval_texts = eval_dataset['text_query'][:num_samples]
    true_commands = eval_dataset['pandas_code'][:num_samples]

    generated_commands = []

    for text in tqdm(eval_texts):
        inputs = tokenizer(text, return_tensors="pt", max_length=256, truncation=True, padding="max_length").to(device)
        outputs = model.generate(**inputs, **generation_args)
        decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
        generated_commands.append(decoded)

    return eval_texts, true_commands, generated_commands

eval_texts, true_commands, generated_commands = generate_predictions(tokenized_eval_dataset, model, tokenizer, device)

100%|██████████| 100/100 [00:37<00:00,  2.70it/s]


## Compare Predictions

In [ ]:
# Compare the generated commands with the true commands
for i in range(5):  # Display the first 5 for brevity
    print(f"Text Query: {eval_texts[i]}")
    print(f"True Command: {true_commands[i]}")
    print(f"Generated Command: {generated_commands[i]}")
    print()


Text Query: List the cities which have more than 2 airports sorted by the number of airports.
True Command: df['city'].groupby(df['city']).filter(lambda x: len(x) > 2).sort_values()
Generated Command: df['city_count'] == 2]

Text Query: What is the average credit score for customers who have never taken a loan?
True Command: customer[~customer['cust_id'].isin(loan['cust_id'])]['credit_score'].mean()
Generated Command: df['Credit Score']

Text Query: What are the enrollment and primary conference for the university which was founded the earliest?
True Command: df.loc[df['founded'].idxmin(), ['enrollment', 'primary_conference']]
Generated Command: df['University_id'] == 'first_initiated'].detail()

Text Query: Find the delegates who are from counties with population below 100000.
True Command: (county.join(election, county.County_id == election.District)         .query("Population < 100000")         .Delegate)
Generated Command: df['Counties'] == 100000]

Text Query: Show the names of me

## Calculate Metrics

In [ ]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=e57843a1eb40549cb1766807f543bb93ab400b7ed8f999bd0ec4093839565d25
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
from datasets import load_metric

# Load the BLEU and ROUGE metrics
bleu_metric = load_metric('bleu')
rouge_metric = load_metric('rouge')

# Prepare the data for metrics
references = [[command.split()] for command in true_commands]
predictions = [command.split() for command in generated_commands]

# Calculate BLEU score
bleu_score = bleu_metric.compute(predictions=predictions, references=references)
print(f"BLEU score: {bleu_score['bleu']}")

# Calculate ROUGE score
rouge_score = rouge_metric.compute(predictions=generated_commands, references=true_commands)
print(f"ROUGE score: {rouge_score}")


<ipython-input-9-fd8bd2ce68ac>:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  bleu_metric = load_metric('bleu')
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for bleu contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.2/metrics/bleu/bleu.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.2/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


BLEU score: 0.0
ROUGE score: {'rouge1': AggregateScore(low=Score(precision=0.5753104166666665, recall=0.1892917836102818, fmeasure=0.268712071763923), mid=Score(precision=0.6352619047619047, recall=0.2209460656318014, fmeasure=0.30857014741211275), high=Score(precision=0.6948363095238093, recall=0.2537948852607672, fmeasure=0.3471144227858126)), 'rouge2': AggregateScore(low=Score(precision=0.14532916666666668, recall=0.04381783103539399, fmeasure=0.06387180951880644), mid=Score(precision=0.19428571428571428, recall=0.06290207947404741, fmeasure=0.08939637051349586), high=Score(precision=0.24795654761904756, recall=0.08444591542483702, fmeasure=0.11743257907809487)), 'rougeL': AggregateScore(low=Score(precision=0.5547848214285713, recall=0.18299145506567116, fmeasure=0.2582291578544849), mid=Score(precision=0.6107261904761905, recall=0.21051989509750962, fmeasure=0.2948826057660718), high=Score(precision=0.668289880952381, recall=0.24321070543067624, fmeasure=0.3345132421349245)), 'roug

# Bart Model


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load the tokenizer and model
model_name = "facebook/bart-base"  # or "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Tokenize the dataset with a higher max_length
def preprocess_function(examples):
    inputs = examples['text_query']
    targets = examples['pandas_code']
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")

    # Set up the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=512, truncation=True, padding="max_length")

    model_inputs['labels'] = labels['input_ids']
    return model_inputs

tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True)
tokenized_eval_dataset = eval_dataset.map(preprocess_function, batched=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Map:   0%|          | 0/6427 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3946: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/1607 [00:00<?, ? examples/s]

## Training and Evaluation

In [ ]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

# Set up training arguments with appropriate max length
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    eval_accumulation_steps=10,
    generation_max_length=512,
)

# Initialize the Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset,
)

# Train the model
trainer.train()

# Evaluate the model
eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.996600,0.064956
2,0.071400,0.053354
3,0.063200,0.050311


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file

Evaluation results: {'eval_loss': 0.05031118914484978, 'eval_runtime': 73.3136, 'eval_samples_per_second': 21.92, 'eval_steps_per_second': 2.742, 'epoch': 3.0}


## Save the Trained Model

In [ ]:
# Save the model
model.save_pretrained("trained_model")
tokenizer.save_pretrained("trained_model")

# Download the model files (optional)
!zip -r trained_model.zip trained_model
from google.colab import files
files.download("trained_model.zip")


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


  adding: trained_model/ (stored 0%)
  adding: trained_model/merges.txt (deflated 53%)
  adding: trained_model/vocab.json (deflated 59%)
  adding: trained_model/config.json (deflated 64%)
  adding: trained_model/generation_config.json (deflated 47%)
  adding: trained_model/special_tokens_map.json (deflated 52%)
  adding: trained_model/tokenizer.json (deflated 72%)
  adding: trained_model/model.safetensors (deflated 8%)
  adding: trained_model/tokenizer_config.json (deflated 75%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Further Evaluation

In [ ]:
import torch

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Move the model to the device
model.to(device)


Using device: cuda


BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): Laye

## Generate Predictions

In [ ]:
# Generate predictions for a subset of the evaluation dataset
import numpy as np
from tqdm import tqdm

# Adjust generation parameters
generation_args = {
    "max_length": 512,
    "num_beams": 5,
    "early_stopping": True,
    "no_repeat_ngram_size": 2,
}

def generate_predictions(eval_dataset, model, tokenizer, device, num_samples=100):
    eval_texts = eval_dataset['text_query'][:num_samples]
    true_commands = eval_dataset['pandas_code'][:num_samples]

    generated_commands = []

    for text in tqdm(eval_texts):
        inputs = tokenizer(text, return_tensors="pt", max_length=256, truncation=True, padding="max_length").to(device)
        outputs = model.generate(**inputs, **generation_args)
        decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
        generated_commands.append(decoded)

    return eval_texts, true_commands, generated_commands

eval_texts, true_commands, generated_commands = generate_predictions(tokenized_eval_dataset, model, tokenizer, device)

100%|██████████| 100/100 [00:28<00:00,  3.56it/s]


## Compare Predictions

In [ ]:
# Compare the generated commands with the true commands
for i in range(5):  # Display the first 5 for brevity
    print(f"Text Query: {eval_texts[i]}")
    print(f"True Command: {true_commands[i]}")
    print(f"Generated Command: {generated_commands[i]}")
    print()


Text Query: List the cities which have more than 2 airports sorted by the number of airports.
True Command: df['city'].groupby(df['city']).filter(lambda x: len(x) > 2).sort_values()
Generated Command: df.groupby('city').filter(lambda x: len(x) > 2)['city'].count()

Text Query: What is the average credit score for customers who have never taken a loan?
True Command: customer[~customer['cust_id'].isin(loan['cust_id'])]['credit_score'].mean()
Generated Command: df[~df['credit_score'].isin(df_loan['cust_id'])]['fname']

Text Query: What are the enrollment and primary conference for the university which was founded the earliest?
True Command: df.loc[df['founded'].idxmin(), ['enrollment', 'primary_conference']]
Generated Command: df.loc[df['founded_date'].idxmin(), ['enrollment', 'primary_conference']]

Text Query: Find the delegates who are from counties with population below 100000.
True Command: (county.join(election, county.County_id == election.District)         .query("Population < 100

## Calculate Metrics

In [ ]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=b9f8e81788553542963b66aceb73ac008561bee30b13394c7295695cdc2a2e2c
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
from datasets import load_metric

# Load the BLEU and ROUGE metrics
bleu_metric = load_metric('bleu')
rouge_metric = load_metric('rouge')

# Prepare the data for metrics
references = [[command.split()] for command in true_commands]
predictions = [command.split() for command in generated_commands]

# Calculate BLEU score
bleu_score = bleu_metric.compute(predictions=predictions, references=references)
print(f"BLEU score: {bleu_score['bleu']}")

# Calculate ROUGE score
rouge_score = rouge_metric.compute(predictions=generated_commands, references=true_commands)
print(f"ROUGE score: {rouge_score}")


/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for bleu contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/bleu/bleu.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major r

BLEU score: 0.042877501662835445
ROUGE score: {'rouge1': AggregateScore(low=Score(precision=0.6245451975108227, recall=0.4555764036150887, fmeasure=0.5018142674051391), mid=Score(precision=0.6840128066378065, recall=0.511993329448386, fmeasure=0.5494769308296562), high=Score(precision=0.737614136904762, recall=0.5659346303550232, fmeasure=0.5989918020011146)), 'rouge2': AggregateScore(low=Score(precision=0.3746486747128232, recall=0.2764963943423198, fmeasure=0.2995943244999709), mid=Score(precision=0.4340314913844325, recall=0.3351327866365575, fmeasure=0.3535227659497522), high=Score(precision=0.4969241447835493, recall=0.3975674742683924, fmeasure=0.4130018701585044)), 'rougeL': AggregateScore(low=Score(precision=0.6011555758477631, recall=0.4408900032295792, fmeasure=0.4822672195003384), mid=Score(precision=0.6594461580086581, recall=0.49456783258772724, fmeasure=0.531589878571284), high=Score(precision=0.7189437635281384, recall=0.5528543796590882, fmeasure=0.586330052673956)), 'r

## Qualitative Analysis

In [ ]:
# Manually inspect a few samples to understand the model's performance
for i in range(5):
    print(f"Text Query: {eval_texts[i]}")
    print(f"True Command: {true_commands[i]}")
    print(f"Generated Command: {generated_commands[i]}")
    print("Difference: ")
    for line in difflib.unified_diff(true_commands[i].splitlines(), generated_commands[i].splitlines(), lineterm=''):
        print(line)
    print()


Text Query: List the cities which have more than 2 airports sorted by the number of airports.
True Command: df['city'].groupby(df['city']).filter(lambda x: len(x) > 2).sort_values()
Generated Command: df.groupby('city').filter(lambda x: len(x) > 2)['city'].count()
Difference: 


NameError: name 'difflib' is not defined